In [9]:
from music21 import converter, instrument, note, chord
import os

In [10]:
# audio_teste = os.getcwd() + '/midi_songs/4WeddingsAndAFuneral.mid'
# audio_teste

In [12]:
notes = []
midi = converter.parse('/Users/luizeduardocartolano/Desktop/10.mid')